# 卷积神经网络基础

> 本文参考：
- 伯禹AI：https://www.boyuai.com/elites/course/cZu18YmweLv10OeV
- 动手学深度学习：http://zh.d2l.ai/
- 动手学深度学习 TF 版：https://trickygo.github.io/Dive-into-DL-TensorFlow2.0


内容包含：

介绍卷积神经网络的基础概念，主要是卷积层和池化层，并解释填充、步幅、输入通道和输出通道的含义。


In [1]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.1.0


## 二维卷积层

本节介绍的是最常见的二维卷积层，常用于处理图像数据。

### 二维互相关运算

二维互相关（cross-correlation）运算的输入是一个二维输入数组和一个二维核（kernel）数组，输出也是一个二维数组，其中核数组通常称为卷积核或过滤器（filter）。卷积核的尺寸通常小于输入数组，卷积核在输入数组上滑动，在每个位置上，卷积核与该位置处的输入子数组按元素相乘并求和，得到输出数组中相应位置的元素。图1展示了一个互相关运算的例子，阴影部分分别是输入的第一个计算区域、核数组以及对应的输出。

![二维互相关运算](../img/chapter05/5.1_correlation.svg)

<div align=center>图1 二维互相关运算</div>

下面我们用`corr2d`函数实现二维互相关运算，它接受输入数组`X`与核数组`K`，并输出数组`Y`。

In [2]:
def corr2d(X, K):
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w +1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j].assign(tf.cast(tf.reduce_sum(X[i:i+h, j:j+w] * K), dtype=tf.float32))
    return Y

构造上图中的输入数组`X`、核数组`K`来验证二维互相关运算的输出。

In [3]:
X = tf.constant([[0,1,2], [3,4,5], [6,7,8]])
K = tf.constant([[0,1], [2,3]])
corr2d(X, K)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[19., 25.],
       [37., 43.]], dtype=float32)>

### 二维卷积层

二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏置来得到输出。卷积层的模型参数包括卷积核和标量偏置。

In [4]:
class Conv2D(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
    
    def build(self, kernel_size):
        self.w = self.add_weight(name='w',
                                shape=kernel_size,
                                initializer=tf.random_normal_initializer())
        self.b = self.add_weight(name='b',
                                shape=(1,),
                                initializer=tf.random_normal_initializer())
    def call(self, inputs):
        return corr2d(inputs, self.w) + self.b

下面我们看一个例子，我们构造一张$6 \times 8$的图像，中间4列为黑（0），其余为白（1），希望检测到颜色边缘。我们的标签是一个$6 \times 7$的二维数组，第2列是1（从1到0的边缘），第6列是-1（从0到1的边缘）。


In [6]:
X = tf.Variable(tf.ones((6,8)))
X[:, 2:6].assign(tf.zeros(X[:,2:6].shape))

# 构造一个高和宽分别为1和2的卷积核`K`。当它与输入做互相关运算时，如果横向相邻元素相同，输出为0；否则输出为非0。
K = tf.constant([[1,-1]], dtype = tf.float32)
Y = corr2d(X, K)
print(X)
print(Y)

<tf.Variable 'Variable:0' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>


我们希望学习一个$1 \times 2$卷积层(即学习出上面的卷积层 K)，通过卷积层来检测颜色边缘。

In [7]:
#input_shape = (samples, rows, cols, channels)   tensorflow 中输入为 样本、高、宽、通道数
X = tf.reshape(X, (1,6,8,1))
Y = tf.reshape(Y, (1,6,7,1))

# 虽然我们之前构造了`Conv2D`类，但由于`corr2d`使用了对单个元素赋值（`[i, j]=`）的操作因而无法自动求梯度。
# 下面我们使用tf.keras.layers提供的`Conv2D`类来实现这个例子。
conv2d = tf.keras.layers.Conv2D(1, (1,2))


Y_hat = conv2d(X)
for i in range(10):
    with tf.GradientTape(watch_accessed_variables=False) as g:
        g.watch(conv2d.weights[0])
        Y_hat = conv2d(X)
        l = (abs(Y_hat - Y)) ** 2
        dl = g.gradient(l, conv2d.weights[0])
        lr = 3e-2
        update = tf.multiply(lr, dl)
        updated_weights = conv2d.get_weights()
        updated_weights[0] = conv2d.weights[0] - update
        conv2d.set_weights(updated_weights)  
        
        if (i + 1)% 2 == 0:
            print('batch %d, loss %.3f' % (i + 1, tf.reduce_sum(l)))


batch 2, loss 15.990
batch 4, loss 6.157
batch 6, loss 2.456
batch 8, loss 0.995
batch 10, loss 0.406


学习到的核数组与K相近

In [8]:
tf.reshape(conv2d.get_weights()[0],(1,2))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 1.0590473, -0.9276972]], dtype=float32)>

### 互相关运算与卷积运算

卷积层得名于卷积运算，但卷积层中用到的并非卷积运算而是互相关运算。我们将核数组上下翻转、左右翻转，再与输入数组做互相关运算，这一过程就是卷积运算。由于卷积层的核数组是可学习的，所以使用互相关运算与使用卷积运算并无本质区别。

### 特征图与感受野

二维卷积层输出的二维数组可以看作是输入在空间维度（宽和高）上某一级的表征，也叫特征图（feature map）。影响元素$x$的前向计算的所有可能输入区域（可能大于输入的实际尺寸）叫做$x$的感受野（receptive field）。

以图1为例，输入中阴影部分的四个元素是输出中阴影部分元素的感受野。我们将图中形状为$2 \times 2$的输出记为$Y$，将$Y$与另一个形状为$2 \times 2$的核数组做互相关运算，输出单个元素$z$。那么，$z$在$Y$上的感受野包括$Y$的全部四个元素，在输入上的感受野包括其中全部9个元素。可见，我们可以通过更深的卷积神经网络使特征图中单个元素的感受野变得更加广阔，从而捕捉输入上更大尺寸的特征。

## 填充和步幅

我们介绍卷积层的两个超参数，即填充和步幅，它们可以对给定形状的输入和卷积核改变输出形状。

### 填充

填充（padding）是指在输入高和宽的两侧填充元素（通常是0元素），图2里我们在原输入高和宽的两侧分别添加了值为0的元素。


<div align=center>
<img  src="../img/chapter05/5.2_conv_pad.svg"/>
</div>

<div align=center>图2 在输入的高和宽两侧分别填充了0元素的二维互相关计算</div>


如果原输入的高和宽是$n_h$和$n_w$，卷积核的高和宽是$k_h$和$k_w$，在高的两侧一共填充$p_h$行，在宽的两侧一共填充$p_w$列，则输出形状为：


$$
(n_h+p_h-k_h+1)\times(n_w+p_w-k_w+1)
$$


我们在卷积神经网络中使用奇数高宽的核，比如$3 \times 3$，$5 \times 5$的卷积核，对于高度（或宽度）为大小为$2 k + 1$的核，令步幅为1，在高（或宽）两侧选择大小为$k$的填充，便可保持输入与输出尺寸相同。

### 步幅

在互相关运算中，卷积核在输入数组上滑动，每次滑动的行数与列数即是步幅（stride）。此前我们使用的步幅都是1，图3展示了在高上步幅为3、在宽上步幅为2的二维互相关运算。


![高和宽上步幅分别为3和2的二维互相关运算](../img/chapter05/5.2_conv_stride.svg)

<div align=center>图3 高和宽上步幅分别为3和2的二维互相关运算</div>


一般来说，当高上步幅为$s_h$，宽上步幅为$s_w$时，输出形状为：


$$
\lfloor(n_h+p_h-k_h+s_h)/s_h\rfloor \times \lfloor(n_w+p_w-k_w+s_w)/s_w\rfloor
$$


如果$p_h=k_h-1$，$p_w=k_w-1$，那么输出形状将简化为$\lfloor(n_h+s_h-1)/s_h\rfloor \times \lfloor(n_w+s_w-1)/s_w\rfloor$。更进一步，如果输入的高和宽能分别被高和宽上的步幅整除，那么输出形状将是$(n_h / s_h) \times (n_w/s_w)$。

当$p_h = p_w = p$时，我们称填充为$p$；当$s_h = s_w = s$时，我们称步幅为$s$。

## 多输入通道和多输出通道

之前的输入和输出都是二维数组，但真实数据的维度经常更高。例如，彩色图像在高和宽2个维度外还有RGB（红、绿、蓝）3个颜色通道。假设彩色图像的高和宽分别是$h$和$w$（像素），那么它可以表示为一个$3 \times h \times w$的多维数组，我们将大小为3的这一维称为通道（channel）维。

### 多输入通道

卷积层的输入可以包含多个通道，图4展示了一个含2个输入通道的二维互相关计算的例子。


<div align=center>
<img  src="../img/chapter05/5.3_conv_multi_in.svg"/>
</div>
<div align=center>图4 含2个输入通道的互相关计算</div>

假设输入数据的通道数为$c_i$，卷积核形状为$k_h\times k_w$，我们为每个输入通道各分配一个形状为$k_h\times k_w$的核数组，将$c_i$个互相关运算的二维输出按通道相加，得到一个二维数组作为输出。我们把$c_i$个核数组在通道维上连结，即得到一个形状为$c_i\times k_h\times k_w$的卷积核。

### 多输出通道

卷积层的输出也可以包含多个通道，设卷积核输入通道数和输出通道数分别为$c_i$和$c_o$，高和宽分别为$k_h$和$k_w$。如果希望得到含多个通道的输出，我们可以为每个输出通道分别创建形状为$c_i\times k_h\times k_w$的核数组，将它们在输出通道维上连结，卷积核的形状即$c_o\times c_i\times k_h\times k_w$。

对于输出通道的卷积核，我们提供这样一种理解，一个$c_i \times k_h \times k_w$的核数组可以提取某种局部特征，但是输入可能具有相当丰富的特征，我们需要有多个这样的$c_i \times k_h \times k_w$的核数组，不同的核数组提取的是不同的特征。

### 1x1卷积层

最后讨论形状为$1 \times 1$的卷积核，我们通常称这样的卷积运算为$1 \times 1$卷积，称包含这种卷积核的卷积层为$1 \times 1$卷积层。图5展示了使用输入通道数为3、输出通道数为2的$1\times 1$卷积核的互相关计算。

<div align=center>
<img  src="../img/chapter05/5.3_conv_1x1.svg"/>
</div>
<div align=center>图5 使用输入通道数为3、输出通道数为2的$1\times 1$卷积核的互相关计算。输入和输出具有相同的高和宽</div>

$1 \times 1$卷积核可在不改变高宽的情况下，调整通道数。$1 \times 1$卷积核不识别高和宽维度上相邻元素构成的模式，其主要计算发生在通道维上。假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么$1\times 1$卷积层的作用与全连接层等价。

## 卷积层与全连接层的对比

二维卷积层经常用于处理图像，与此前的全连接层相比，它主要有两个优势：

一是全连接层把图像展平成一个向量，在输入图像上相邻的元素可能因为展平操作不再相邻，网络难以捕捉局部信息。而卷积层的设计，天然地具有提取局部信息的能力。

二是卷积层的参数量更少。不考虑偏置的情况下，一个形状为$(c_i, c_o, h, w)$的卷积核的参数量是$c_i \times c_o \times h \times w$，与输入图像的宽高无关。假如一个卷积层的输入和输出形状分别是$(c_1, h_1, w_1)$和$(c_2, h_2, w_2)$，如果要用全连接层进行连接，参数数量就是$c_1 \times c_2 \times h_1 \times w_1 \times h_2 \times w_2$。使用卷积层可以以较少的参数数量来处理更大的图像。

## 卷积层的简洁实现

我们使用Tensorflowh中的`tf.keras.layers.Conv2D`类来实现二维卷积层，主要关注以下几个构造函数参数：

* `filters`  filters: Integer, the dimensionality of the output space
* `kernel_size` (python:int or tuple, optional)  height and width of the 2D convolution window, default (1, 1)
* `strides` (python:int or tuple, optional) – specifying the strides of the convolution along the height and width.
* `padding` valid:表示不够卷积核大小的块,则丢弃;same 表示不够卷积核大小的块就补0,默认 valid
* `use_bias`  Boolean, whether the layer uses a bias vector. default True



In [21]:
#tensorflow default data_format == 'channels_last' 
#input_shape = (samples, rows, cols, channels)
X = tf.random.uniform((4,2,3,2))
conv2d = tf.keras.layers.Conv2D(filters=5, kernel_size=(2,3), strides=1, padding='same')
Y=conv2d(X)
print(X.shape)
print(Y.shape)

(4, 2, 3, 2)
(4, 2, 3, 5)


## 池化

### 二维池化层

池化层主要用于缓解卷积层对位置的过度敏感性。实际图像里，我们感兴趣的物体不会总出现在固定位置：即使我们连续拍摄同一个物体也极有可能出现像素位置上的偏移。同卷积层一样，池化层每次对输入数据的一个固定形状窗口（又称池化窗口）中的元素计算输出，池化层直接计算池化窗口内元素的最大值或者平均值，该运算也分别叫做最大池化或平均池化。图6展示了池化窗口形状为$2\times 2$的最大池化。


<div align=center>
<img  src="../img/chapter05/5.4_pooling.svg"/>
</div>
<div align=center>图6 池化窗口形状为$2\times 2$的最大池化</div>

二维平均池化的工作原理与二维最大池化类似，但将最大运算符替换成平均运算符。池化窗口形状为$p \times q$的池化层称为$p \times q$池化层，其中的池化运算叫作$p \times q$池化。

池化层也可以在输入的高和宽两侧填充并调整窗口的移动步幅来改变输出形状。池化层填充和步幅与卷积层填充和步幅的工作机制一样。

在处理多通道输入数据时，池化层对每个输入通道分别池化，但不会像卷积层那样将各通道的结果按通道相加。这意味着池化层的输出通道数与输入通道数相等。

### 池化层的简洁实现

我们使用Tensorflowh中的`tf.keras.layers.MaxPool2D`类来实现最大池化层，主要关注以下几个构造函数参数：

* `pool_size` (python:int or tuple, optional)  height and width of the 2D convolution window.
* `strides` (python:int or tuple, optional) – specifying the strides of the convolution along the height and width， default pool_size
* `padding` valid:表示不够卷积核大小的块,则丢弃;same 表示不够卷积核大小的块就补0,所以输出和输入形状相同，默认 valid
* `use_bias`  Boolean, whether the layer uses a bias vector. default True


In [23]:
X = tf.reshape(tf.constant(range(32)), (1,4,4,2))
pool2d = tf.keras.layers.MaxPool2D(pool_size=[3,3],padding='same',strides=(2,1))
Y=pool2d(X)
print(X)
print(Y)

tf.Tensor(
[[[[ 0  1]
   [ 2  3]
   [ 4  5]
   [ 6  7]]

  [[ 8  9]
   [10 11]
   [12 13]
   [14 15]]

  [[16 17]
   [18 19]
   [20 21]
   [22 23]]

  [[24 25]
   [26 27]
   [28 29]
   [30 31]]]], shape=(1, 4, 4, 2), dtype=int32)
tf.Tensor(
[[[[18 19]
   [20 21]
   [22 23]
   [22 23]]

  [[26 27]
   [28 29]
   [30 31]
   [30 31]]]], shape=(1, 2, 4, 2), dtype=int32)


平均池化层使用的是`tf.keras.layers.AveragePooling2D`，使用方法与`tf.keras.layers.MaxPool2D`相同。